## ready_array2.npz as dataset

In [1]:
import numpy as np
import pandas as pd

In [26]:
cholesterol = 'C:/Users/CCheny/Documents/UC San Diego - Junction/Bioinformatics/MS2-Autoencoder/Output/cholesterol/ready_array2.npz'

In [27]:
file = np.load(cholesterol, allow_pickle=True)
data = file['arr_0']

In [28]:
np.random.shuffle(data)
new_list = np.split(data, 2, axis=1)

In [9]:
data3 = new_list[0]
print(data3.shape)
data3 = data3.reshape(data3.shape[0], data3.shape[2], data3.shape[3])
data3[:,:,0] = data3[:,:,0].astype('float64') / 2000
data3[:,:,1] = data3[:,:,1].astype('float64') / 127984
print(data3.shape)
data4 = new_list[1]
data4[:,:,0] = data4[:,:,0].astype('float64') / 2000
data4[:,:,1] = data4[:,:,1].astype('float64') / 127984
data4 = data4.reshape(data4.shape[0], data4.shape[2], data4.shape[3])

(5999, 1, 2000, 2)
(5999, 2000, 2)


In [37]:
low_peaks = new_list[0]
low_peaks = low_peaks.astype('float32') / np.amax(data)
print(low_peaks.shape)
high_peaks = new_list[1]
high_peaks = high_peaks.astype('float32') / np.amax(data)
print(high_peaks.shape)

(5999, 1, 2000)
(5999, 1, 2000)


## keras

In [38]:
from keras.layers import Input, Dense
from keras.models import Model

In [39]:
X = low_peaks
y = high_peaks

In [40]:
X.shape

(5999, 1, 2000)

In [41]:
Xcut = int(0.8*len(X))
X_train = X[:Xcut,:,:]
X_test = X[Xcut:,:,:]
print(X_train.shape)
print(X_test.shape)
X_train = X_train.reshape(len(X_train), np.prod(X_train.shape[1:]))
X_test = X_test.reshape(len(X_test), np.prod(X_test.shape[1:]))
print(X_train.shape)
print(X_test.shape)

ycut = int(0.8*len(y))
y_train = y[:ycut,:,:]
y_test = y[ycut:,:,:]
print(y_train.shape)
print(y_test.shape)
y_train = y_train.reshape(len(y_train), np.prod(y_train.shape[1:]))
y_test = y_test.reshape(len(y_test), np.prod(y_test.shape[1:]))
print(y_train.shape)
print(y_test.shape)

(4799, 1, 2000)
(1200, 1, 2000)
(4799, 2000)
(1200, 2000)
(4799, 1, 2000)
(1200, 1, 2000)
(4799, 2000)
(1200, 2000)


In [46]:
encoding_dim = 100
input_scan = Input(shape=(2000,))
encoded = Dense(encoding_dim, activation='softplus')(input_scan)
decoded = Dense(2000, activation='sigmoid')(encoded)

autoencoder = Model(input_scan, decoded)
encoder = Model(input_scan, encoded)

In [47]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [48]:
autoencoder.compile(optimizer='adadelta', loss='poisson')

In [49]:
autoencoder.fit(X_train, y_train, 
                epochs=50, 
                batch_size=200, 
                validation_data=(X_test, y_test))

Train on 4799 samples, validate on 1200 samples
Epoch 1/50
4799/4799 [==============================] - 1s 139us/step - loss: 0.4816 - val_loss: 0.4629
Epoch 2/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.4457 - val_loss: 0.4271
Epoch 3/50
4799/4799 [==============================] - 0s 61us/step - loss: 0.4101 - val_loss: 0.3916
Epoch 4/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.3749 - val_loss: 0.3567
Epoch 5/50
4799/4799 [==============================] - 0s 62us/step - loss: 0.3404 - val_loss: 0.3228
Epoch 6/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.3070 - val_loss: 0.2901
Epoch 7/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.2751 - val_loss: 0.2591
Epoch 8/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.2451 - val_loss: 0.2302
Epoch 9/50
4799/4799 [==============================] - 0s 59us/step - loss: 0.2173 - val_loss: 0.2037
Epoch 10/50
4799/4799 [=

In [50]:
encoded_scans = encoder.predict(X_test)
decoded_scans = decoder.predict(encoded_scans)

In [52]:
decoded_scans.reshape(1200, 2000)

array([[0.01006466, 0.01047942, 0.00987396, ..., 0.01028851, 0.01042628,
        0.0107803 ],
       [0.01006618, 0.0104804 , 0.0098761 , ..., 0.01029208, 0.01042759,
        0.01078603],
       [0.0100629 , 0.01047912, 0.00987396, ..., 0.01028797, 0.01042759,
        0.01078111],
       ...,
       [0.01006225, 0.01047936, 0.00987437, ..., 0.01028663, 0.01042736,
        0.01078096],
       [0.01006028, 0.01047701, 0.0098713 , ..., 0.01028514, 0.01042256,
        0.01077637],
       [0.01006073, 0.01047853, 0.00987342, ..., 0.01028568, 0.01042542,
        0.01077908]], dtype=float32)

In [53]:
X_test.reshape(1200, 2000)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)